# HWK6 Part 2: Geovisualization

#### Name: Dylan Lam
#### UT EID: DXL85

#### Name: Eric Nguyen
#### UT EID: edn447

#### Date:

In this portion of the homework we will be analyzing and visualizing geographic data; i.e., data that refers to geospatial entities. Geospatial entities can, for example, be particular places such as schools and libraries or political boundaries of cities or countries. Of course, this tutorial only scratches the surface. Consider this as a teaser into geovisualization, which in itself has become a branch of research and practice at the intersection of geography and visualization.

In [1]:
import altair as alt
import pandas as pd

### Install packages

For this tutorial we will continue to rely on Altair and Pandas, but add **GeoPandas**, which will help us to work with DataFrames that contain spatial entities to carry out geometric analysis on them. As before, the pip install command is carried out via the shell, which is indicated by the exclamation mark at the beginning of the line:

In [2]:
!pip install geopandas
import geopandas as gpd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 16.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00


To access the data of the OpenStreetMap, we will install the handy package **OSMPythonTools**:

In [3]:
!pip install --upgrade OSMPythonTools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.4 MB/s eta 0:00:00a 0:00:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.1/973.1 kB 11.9 MB/s eta 0:00:00
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.3.5-py3-none-any.whl size=32715 sha256=0555e408940582b48264b5354424fea05ed58684b9350849b9977e2fb8323de4
  Stored in directory: /Users/dylanlam/Library/Caches/pip/wheels/dc/2e/0b/51582d4741d2847aa6b6cac475561cc83ee65337380d88fb8c
  Created wheel for lxml: filename=lxml-5.3.0-cp38-cp38-macosx_11_0_arm64.whl size=1556169 sha256=05e12c55ed06837202f426fc24440f654f8a2364cea6994d1865afa5291212d4
  Stored in directory: /Users/dylanlam/Library/Caches/pip/wheels/37/19/50/80ff9642d74365e416a0b370a5784afd3167f4e1182644c30b
Successfully built OSMPythonTools lxml


Once we have the tools assembled, we can get started working with geospatial data. There are actually plenty of formats used to record geospatial data, but GeoJSON has become an important standard for exchanging geospatial data on the web. However, please note that GeoPandas can actually load many other vector-based data formats used in digital cartography, such as shapefiles and GeoPackage.

### Import GeoJSON

Suppose we would like to get the geographic boundaries of Austin's neighborhoods. Akin to how we would read a JSON file with Pandas, we can also use `read_file()` provided by GeoPandas simply by passing the geojson filename and get a geographic DataFrame back:

In [6]:
neighborhoods = gpd.read_file("https://gist.githubusercontent.com/TieJean/40e9ccc69f0cc65b8e06ccf3fd60a3f0/raw/04ec5f998e63185e65f4d415827f1838ff8a25ab/austin.geojson")

AttributeError: module 'fiona' has no attribute 'path'

The main difference between a regular Pandas DataFrame is that a GeoDataFrame features a `geometry` column, which is a geoseries containing the points, paths, and polygons for each row. For example, if each row represents one neighborhood, the respective geometries would probably contain the geospatial boundaries…

💡 *Are you curious what the neighborhoods dataframe actually looks like? Take a look at it with the methods you know by now:* 

In [ ]:
neighborhoods.head()

Geographically speaking, the districts are defined by their geographic shapes, which are represented as polygons, each of which is a list of tuples of geographic coordinates. Next we add information about schools in Austin:

In [ ]:
schools = gpd.read_file("https://gist.githubusercontent.com/TieJean/4e9f595eb3cb5dc60a960c3a499f6141/raw/171118ec54c599ce4bf9a30f9ab176427a47629f/austin_schools.geojson")
schools.head()

💡 *Have a look at the schools as well, and compare the contents of the `geometry` columns in schools and districts. Do you notice anything?*

### Query OpenStreetMap

OpenStreetMap (OSM) is "a collaborative project to create a free editable map of the world". As such it has millions of contributing users who have been collecting, updating and refining map data for over 15 years, which has generated a vastly comprehensive source of geographic data. It is by no means complete—whatever this would mean—but it is an impressively large geographic database and, of course, a map in itself, too.

OpenStreetMap has its own kind of query language, which is quite compact and can also be a source for errors. To make query formulation easier, you can either use the web interface [overpass turbo](http://overpass-turbo.eu) or the `overpassQueryBuilder`, which provides access to the main parameters:

In [ ]:
from OSMPythonTools.overpass import overpassQueryBuilder

AustinAreaID = 3600000000 + 113314
# The area ID of a city is found by adding 3600000000 to the city's relation ID
# You can look up the relation ID of any city by searching on the OSM website; for example, https://www.openstreetmap.org/relation/113314 

library_query = overpassQueryBuilder(
    area=AustinAreaID, # the query can be constrained by an area of an item
    elementType='node', # which are points (OSM also has ways and relations)
    # the selector in the next line is really the heart of the query:
    selector='"amenity"="library"', # we're looking for libraries
    out='body', # body indicates that we want the data, not just the count
    includeGeometry=True # and we want the geometric information, too
)

library_query

The output of above cell is the compact version of the query, which is carried out in the next step:



In [ ]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()

lib_data = overpass.query(library_query)

The variable `lib_data` now already contains the result from the query against OSM. Let's have a look at it. With `nodes()` we can access the retrieved points. Let's take a look at the first entry:

In [ ]:
lib_data.nodes()[0].tags()

Similarly, we can also access the geometry, which in this case is just a point:

In [ ]:
lib_data.nodes()[0].geometry()

Next, we use the compact form of a list comprehension to extract the libraries' names and coordinates:


In [ ]:
libraries = [ (lib.tag("name"), lib.geometry() ) for lib in lib_data.nodes()]

… which we turn into a GeoDataFrame. By naming the second column `geometry` we indicate towards GeoPandas to interpret the coordinates as geographic locations:

In [ ]:
libraries = gpd.GeoDataFrame(libraries, columns = ['name', 'geometry'])
libraries.head()

## 🗺  Present

When we have geospatial data readily available as GeoDataFrames, we can now map them with Altair. 

(There are other mapping libraries for Python, such as [Folium](https://python-visualization.github.io/folium/), that provide additional functionalities. Altair's geovis features are basic, but do provide some variety of techniques and have the benefit to work consistently with the other chart types we covered.)


### Markers on maps

A simple start is placing locations on a base map and adding a bit of further information via tooltips. Let's do this with Austin's schools! First, we can have another look at the attributes:

In [ ]:
schools.head()

We will now create a simple map with markers in the form of an  Altair chart consisting of two layers:

1.   The `neighborhoods` form the lower layer representing their boundaries and the overall geographic shape of Austin
2.   The `schools` are the points of interests that are displayed on top

When putting the two layers together they should actually refer to the same geographic location to make sense. Here the neighborhoods and schools both refer to Austin. Also note that the order when the charts are added together determines the vertical order: first the basemap and then markers on top.

In [ ]:
# 1.  mark_geoshape transparently uses the geometry column
basemap = alt.Chart(neighborhoods).mark_geoshape(
    # add some styling to reduce the salience of the basemap
    fill="lightgray", stroke="darkgray",
).encode(
    tooltip = ['name'],
).properties(width=600, height=600)

# 2.  we use mark_circle to have more control over visual variables
markers = alt.Chart(schools).mark_circle(opacity=1).encode(
    # point latitude & longitude to coordinates in geometry column
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['NAME', 'STREET', 'ZIP'],
)

# combine the two layers 
basemap + markers

### Dot density maps

Let's use the open maps data set again, and plot New York's trees. Note, we will have to disable the max rows since there are more than 5,000 trees returned from the query.

In [ ]:
NewYorkCityAreaID = 3600000000 + 175905

# 1. prepare query (and directly include the location lookup)
tree_query = overpassQueryBuilder(
    area=NewYorkCityAreaID,
    elementType='node',
    selector='"natural"="tree"', 
    out='body', 
    includeGeometry=True
)

# 2. execute query (and give it a bit more time to finish)
tree_data = overpass.query(tree_query, timeout=60)

# 3. get ids and coordinates of trees
tree_locs = [ (tree.id(), tree.geometry()) for tree in tree_data.nodes()]

# 4. create GeoDataFrame
trees = gpd.GeoDataFrame(tree_locs, columns=["id", "geometry"])

trees.head()

In [ ]:
alt.data_transformers.disable_max_rows()
treemap = alt.Chart(trees).mark_circle(
    # reduce the visual presence of each element
    size=5,
    # with a low dot opacity we can use overplotting to indicate densities
    opacity=.25,
    # a natural choice
    color="green"
).encode(
    longitude='geometry.coordinates[0]:Q', 
    latitude='geometry.coordinates[1]:Q'    
).properties(width=600, height=600)

treemap

### Choropleth maps

Finally, let's create the geovisualization that uses the fill color of geospatial shapes to encode quantitative data. To illustrate this, we will visualize the population densities around the world. We will use area and population information from GeoNames and get the geographic shapes of countries from a geojson file.

In [ ]:
# load country data from geonames CSV
geonames = pd.read_csv("./countryInfoCSV.csv", sep='\t')
# select four columns
geonames = geonames[['name', 'iso alpha3', 'areaInSqKm', 'population']]
# set index to country code
geonames = geonames.set_index("iso alpha3")

geonames.head()

Next we collect the geographic boundaries and `simplify` them a bit, as they have more detail than what we need here:

In [ ]:
# load country's polygons from datahub
polygons = gpd.read_file("https://gist.githubusercontent.com/TieJean/f739f67075108868059b101a709a738f/raw/cb358884e028658993c2da1dfd19854e6c5b6c3b/countries.geo.json")
# remove country names, as we have them already
polygons = polygons.drop(columns=["name"])
# set index to country code
polygons = polygons.set_index("id")
# reduce the complexity of the shapes
polygons.geometry = polygons.geometry.simplify(.1)

polygons.head(20)

As both DataFrames use the three-letter country codes as indices we can join them like this (join uses the index by default, so we don't have to specify what to join on):

In [ ]:
# inner means that we keep only those countries
# for which we have geometric and attribute data
countries = polygons.join(geonames, how='inner')

countries.tail()

Visualizing area or population in a choropleth map, arguably, makes little sense. So let's compute population densities:

In [ ]:
countries["density"] = countries["population"] / countries["areaInSqKm"]
countries.head()

Keep only those countries with valid density value and turn these densities into integers:

In [ ]:
countries = countries[countries['density'].notna()]
countries.density = countries.density.round(0).astype(int)

There is one ‘country’ that is not really one, which is Antarctica. We'll remove this from the list here.

In [ ]:
countries = countries.drop("ATA")

Finally, we draw the chart using Altair's `mark_geoshape()` method. The distribution of densities is highly skewed, due to very small countries with relatively high population numbers, such as Monaco. To spread out the low and high density values we use a logarithmic scale and set the domain between 1 and 1000. Note that the domain has to end in a multiple of the base, which is by default 10.

In [ ]:
alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('density', scale=alt.Scale(type="log", domain=[1,1000] )),
    tooltip=['name', 'areaInSqKm', 'population', 'density']
).properties(
    width=800,
    height=600
)

The map is shown in the default Mercator projection, which particularly distorts the area sizes of North America, Europe and Russia in contrast to Africa, Southern Asia and parts of South America.

💡 *You can change the projection used above to one that does not distort area sizes as much ([see this list for options](https://vega.github.io/vega-lite/docs/projection.html#projection-types)).* 

## Your Turn

**Q1 (10 points).** Create a visualization with Austin's neighborhoods overlayed with Austin's libraries. The tool tip should provide necessary information to identify each neighborhood and library.

In [5]:
df

NameError: name 'df' is not defined

**Q2 (5 points).** Do you think the open data source for Austin's libraries is reliable? Why or why not? Answer in the Markdown cell below.

**Q3 (10 points).** Add the location of all the trees in Austin (according to Open Street Map) to the visualization you just created in Q1.

**Q4 (10 points).** Create a global population density choropleth with the Albers map projection.

## Sources

Tutorials & Documentation
- [Specifying Geospatial Data in Altair — Altair 4.1.0 documentation](https://altair-viz.github.io/user_guide/data.html#geospatial-data)
- [GeoPandas](https://geopandas.org)
- [OSMPythonTools](https://github.com/mocnik-science/osm-python-tools)

Data
- [OpenStreetMap](https://www.openstreetmap.org/)
- [GeoNames](https://www.geonames.org)
- [Data.gov](https://catalog.data.gov/dataset/public-school-locations-current)
